In [31]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import radon
import torch.nn as nn
import torch.functional as F

In [32]:
train_data = datasets.MNIST(root = './data', train = True, download = True).data / 255
test_data = datasets.MNIST(root = './data', train = False, download = True).data / 255

## -----------------------Data Preprocessing --------------------------

## Radon matrix construction

In [33]:
# Radon Matrix exists for every angle and every line. matrix 
def getRadonMatrix(theta,ny,nx):
    colum_of_A = radon(np.zeros((ny,nx)), theta = theta) # this is only to know the size
    A = np.zeros((colum_of_A.size, nx*ny))
    for i in range(ny):
        for j in range(nx):
            basis_vec = np.zeros((ny,nx))
            basis_vec[i,j] = 1
            colum_of_A = radon(basis_vec, theta = theta)
            A[:,j+i*nx] = np.reshape(colum_of_A, colum_of_A.size)
    return A.do

In [ ]:
train_data.shape

## class for Noisy Sinogram vector using Radon matrix

In [61]:
class sinogram(object):
    def __call__(self, image_data):
        
        ny, nx = image_data.shape[1], image_data.shape[2]
        # theta values for X ray incidence angles
        theta = np.linspace(0,180,14,endpoint = True)
        A = getRadonMatrix(theta, ny, nx)
        sino_gram = image_data @ A.T
        noise_level = 0.01 
        sino_gram += noise_level * np.random.normal(sino_gram.shape)
        
        return sino_gram.double()

In [62]:
sinogram()(train_data)

C:\Users\asvp0\Anaconda3\envs\dl21\lib\site-packages\skimage\transform\radon_transform.py:75: UserWarning: Radon transform: image must be zero outside the reconstruction circle
  warn('Radon transform: image must be zero outside the '


AttributeError: 'numpy.ndarray' object has no attribute 'double'

## class to Vectorize the image data 

In [35]:
class vectorize_image(object):
    def __call__(self, image_data):
        assert type(image_data)==numpy
        
        return np.reshape(image_data, (image_data.shape[0],image_data.shape[1]*image_data.shape[2]))

## class to convert numpy to Tensor

In [36]:
class ToTensor(object):
    def __call__(self, image_data):

        return torch.from_numpy(image_data)  

## Transforms

In [51]:
train_transforms = transforms.Compose([
                    #vectorize_image(),
                    #sinogram(),
                    #ToTensor()
                    ])

## Custom Dataset class using torch.utils.data.Dataset

In [52]:
class Processed_Data(Dataset):
    def __init__(self,data,transform = train_transforms()):
        self.transforms = transform
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        train_data_in_a_index = self.data[idx]
        sinogram_data_in_a_index = self.transforms(train_data_in_a_index)

        return sinogram_data_in_a_index, ToTensor(train_data_in_a_index)

TypeError: __call__() missing 1 required positional argument: 'img'

In [42]:
train_ds = Processed_Data(train_data)
valid_ds = Processed_Data(test_data)

NameError: name 'Processed_Data' is not defined

## Dataloader to load in batches

In [ ]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)